## Importing required libraries

In [ ]:
import os

import pandas as pd
import numpy as np

from PIL import Image

import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
print(os.listdir("../input"))

## Checking the gpu availability

In [ ]:
# gpu test
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Loading the training csv

In [ ]:
# lets load the train.csv
train_df = pd.read_csv("../input/train.csv")
le = LabelEncoder()
train_df['target'] = le.fit_transform(train_df['Id'])
train_df.head()

## Visualizing images

In [ ]:
# lets plot some images
fig = plt.figure(figsize=(25, 10))
for idx, img in enumerate(np.random.choice(os.listdir("../input/train"), 10)):
    ax = fig.add_subplot(2, 10//2, idx+1, xticks=[], yticks=[])
    im = Image.open("../input/train/" + img)
    plt.imshow(im)
    lab = train_df.loc[train_df.Image == img, 'Id'].values[0]
    ax.set_title(f'Label: {lab}')

## Prearing data set for pytorch data loader
Pytorch does required image classes in different folders. But this data set contains all images in one folder. So we need to customise Dataset class of pytorch.

In [ ]:
class HW_Dataset(Dataset):
    def __init__(self,filepath,train_df,transform=None):
        self.file_path = filepath
        self.df = train_df
        self.transform = transform
        self.image_list = [x for x in os.listdir(self.file_path)]
        
    def __len__(self):
        return(len(self.image_list))
    
    def __getitem__(self,idx):
        img_path = os.path.join(self.file_path,self.df.Image[idx])
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        target = self.df.target[idx]
        return (img,target)

## Data loading to Data loader

In [ ]:
transform = transforms.Compose([transforms.RandomResizedCrop(224), 
                                transforms.RandomHorizontalFlip(), 
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])

train_dataset = HW_Dataset(filepath='../input/train/',train_df=train_df,transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, num_workers=0)

## Pretrained Model

In [ ]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

## Model building

In [ ]:
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('do1', nn.Dropout(0.2)),
                          ('fc1', nn.Linear(512, 5005))
                          ]))
    
model.fc = classifier

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

## Training

In [ ]:
n_epochs = 30
valid_loss_min = np.Inf

for epoch in range(1, n_epochs+1):
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    train_acc = 0.0
    valid_acc = 0.0
    ###################
    # train the model #
    ###################
    model.train()
    
    scheduler.step()
    running_loss = 0.0
    running_corrects = 0

    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        #print(target.data)
        data, target = data.to(device), target.to(device)
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        _, preds = torch.max(output, 1)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)
    print('Epoch: {} \t{:.6f} \t {:.0%}'.format( epoch, epoch_loss, epoch_acc))